In [1]:
import pickle
import numpy as np

In [2]:
with open("../input/midi-processed-file/notes", "rb") as f:
    notes = pickle.load(f)

In [3]:
n_vocab = len(set(notes))

In [4]:
print("Total notes :", len(notes))
print("Unique notes :" ,n_vocab)

Total notes : 62377
Unique notes : 374


## Preparing Sequential Data for LSTM Model

In [5]:
# Chosing the sequence length
# i.e we will give 100 elements to the LSTM input for consideration/prediction
sequence_length = 100


In [6]:
# CLASSES
pitchnames = sorted(set(notes))

In [7]:
# Converting this String Data into Numbers
# Therefore, we will create a mapping b/w elements

ele_to_int = dict( (ele, num) for num, ele in enumerate(pitchnames) )
int_to_ele = dict( (num, ele) for num, ele in enumerate(pitchnames) )

In [8]:
network_input = []
network_output = []

In [9]:
# AIM :
#     Input         :      Output
# 1st 100 elements  :  101th element
# 2-101 elements    :  102th element
# 3-102 elements    :  103th element
# and so on...

for i in range(len(notes) - sequence_length):
    
    seq_in = notes[i: i+sequence_length] # Contains 100 values
    seq_out = notes[i+sequence_length]
    
    # These are notes
    # But Neural Networks work with Numeric Data
    # Converting and Appending them
    network_input.append([ ele_to_int[ch] for ch in seq_in])
    network_output.append(ele_to_int[seq_out])

In [10]:
# Number of exmaples
n_patterns = len(network_input)
print(n_patterns)

62277


In [11]:
# Reshaping for efficient training
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
print(network_input.shape)

(62277, 100, 1)


In [12]:
# Normalize the Data
# Bring in the range [0-1]
normalized_network_input = network_input/float(n_vocab)

In [13]:
# Converting the output into one hot encoding
from keras.utils import np_utils
network_output = np_utils.to_categorical(network_output)

In [14]:
network_output.shape

(62277, 374)

## Preprocessed Data for the Model

In [15]:
print(normalized_network_input.shape)
print(network_output.shape)

(62277, 100, 1)
(62277, 374)


# __Model Preparation__

In [16]:
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [17]:
# Model Architecture

model = Sequential()
model.add(LSTM(units=512,
              input_shape= (normalized_network_input.shape[1], normalized_network_input.shape[2]),
              return_sequences = True))
model.add(Dropout(0.3))
model.add(LSTM( 512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM( 512 ))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab, activation='softmax'))


In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 512)          1052672   
_________________________________________________________________
dropout (Dropout)            (None, 100, 512)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

In [20]:
checkpoint = ModelCheckpoint('trained_model_gpu.hdf5', monitor='loss', save_best_only=True, mode='min', verbose=0)

In [21]:
model_his = model.fit(normalized_network_input, network_output, epochs=100, batch_size=64, callbacks=[checkpoint])

Epoch 1/100
974/974 [==============================] - 64s 66ms/step - loss: 4.7907
Epoch 2/100
974/974 [==============================] - 64s 66ms/step - loss: 4.7459
Epoch 3/100
974/974 [==============================] - 65s 66ms/step - loss: 4.7409
Epoch 4/100
974/974 [==============================] - 64s 66ms/step - loss: 4.7399
Epoch 5/100
974/974 [==============================] - 65s 66ms/step - loss: 4.7376
Epoch 6/100
974/974 [==============================] - 65s 66ms/step - loss: 4.7374
Epoch 7/100
974/974 [==============================] - 64s 66ms/step - loss: 4.7364
Epoch 8/100
974/974 [==============================] - 65s 66ms/step - loss: 4.7351
Epoch 9/100
974/974 [==============================] - 64s 66ms/step - loss: 4.7344
Epoch 10/100
974/974 [==============================] - 64s 66ms/step - loss: 4.7342
Epoch 11/100
974/974 [==============================] - 64s 66ms/step - loss: 4.7330
Epoch 12/100
974/974 [==============================] - 64s 66ms/step - lo